# Red Neuronal Recurrente (GRU) para generar una melodía
### Instalar midi_player y music21 (pip install midi_player music21)

In [11]:
import os, glob
import random

import numpy as np 
import pandas as pd
import tensorflow as tf

from midi_player import MIDIPlayer
from music21 import converter, instrument, note, chord, stream

from tensorflow.data import Dataset
from tensorflow.keras import losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

## Leer un archivo de ejemplo

In [2]:
midi_file = 'datasets/mozart/mz_311_1.mid'  # Archivo MIDI de ejemplo
MIDIPlayer(midi_file, 460)  # Reproducir el archivo MIDI

### Extraer la notas de todas la melodías (archivos .mid)

In [7]:
notes = []  # Lista para almacenar las notas extraídas

for file in glob.glob("Datasets/mozart/*.mid"):
    midi = converter.parse(file)  
    
    songs = instrument.partitionByInstrument(midi) 

    for part in songs.parts:
        pick = part.recurse()  
        for element in pick:
            if isinstance(element, note.Note):  
                notes.append(str(element.pitch))  
            elif isinstance(element, chord.Chord):  
                notes.append('.'.join(str(n) for n in element.normalOrder)) 
print(notes)

['2.6.9', '9.2', 'G5', '2.6', 'F#5', 'E5', 'F#5', 'A5', '4.7', 'G5', 'F#5', 'G5', 'A5', '6.9', 'A5', 'B5', 'C#6', 'D6', 'A5', 'F#3', 'D4', 'F#5', 'D3', 'D4', 'A5', 'E3', 'G5', 'F#5', 'D4', 'G5', 'A5', 'A3', 'G5', 'C#4', 'F5', '2', 'F#5', 'G5', '2.6', 'F#5', 'E5', 'F#5', 'A5', '4.7', 'G5', 'F#5', 'G5', 'A5', '6.9', 'A5', 'B5', 'C#6', 'D6', 'A5', 'F#3', 'D4', 'F#5', 'D3', 'D4', 'A5', 'E3', 'G5', 'F#5', 'D4', 'G5', 'A5', 'A3', 'G5', 'F#5', 'C#4', 'E5', 'D5', 'D4', 'F#4', 'A4', 'D4', 'F#4', '6.7', 'D4', '6.7', 'F#4', '6.7', '6.7', 'D4', 'F#5', 'E5', 'F#4', 'D5', 'C#5', 'E4', 'G4', 'A4', 'E4', 'G4', 'E5', 'E4', 'F#4', 'G4', 'F#4', 'E4', 'D4', 'C#4', 'B3', 'E5', 'A3', 'C#4', 'A4', 'A3', 'C#4', '7.9', 'A3', '7.9', 'C#4', '7.9', '7.9', 'G3', 'G5', 'F#5', 'C#4', 'E5', 'D5', 'F#3', 'D4', 'A4', 'F#3', 'D4', 'F#5', 'F#3', 'D3', 'E3', 'F#3', 'G3', 'A3', 'B3', 'C#4', 'F#5', 'D4', 'D5', 'E5', 'F#5', 'G5', 'A5', 'B5', 'C#6', 'D6', 'C#6', 'B5', '6.9', 'A5', 'G5', '6.9', 'F#5', 'E5', '6.9', 'D5', 'E5', 

### Crear un vocabulario de todas las notas encontradas

In [8]:
vocab = sorted(set(notes))
note2idx = {n: i for i, n in enumerate(vocab)}
idx2note = np.array(vocab)

print('Vocabulario: ', len(vocab), ' notas únicas')
print(f'Notas: {vocab}')

Vocabulario:  229  notas únicas
Notas: ['0', '0.1', '0.1.3', '0.2', '0.2.6', '0.2.7', '0.3', '0.3.5', '0.3.6', '0.3.6.9', '0.3.7', '0.4', '0.4.6', '0.4.7', '0.5', '0.6', '1', '1.2', '1.2.4', '1.3', '1.3.7', '1.4', '1.4.7', '1.4.7.10', '1.4.7.9', '1.5', '1.5.8', '1.6', '1.7', '10', '10.0', '10.0.2', '10.0.3', '10.0.4', '10.1', '10.1.4', '10.11', '10.11.1', '10.2', '10.2.3', '10.2.5', '10.3', '11', '11.0', '11.0.2', '11.1', '11.1.2', '11.2', '11.2.4', '11.2.5', '11.2.5.7', '11.2.6', '11.3', '11.3.6', '11.4', '2', '2.3', '2.3.5', '2.4', '2.4.5', '2.4.6', '2.4.8', '2.4.9', '2.5', '2.5.7', '2.5.8', '2.5.8.11', '2.5.9', '2.6', '2.6.9', '2.7', '2.8', '3', '3.4', '3.4.5', '3.4.6', '3.5', '3.5.7', '3.5.9', '3.6', '3.6.9', '3.7', '3.7.10', '3.8', '3.9', '4', '4.10', '4.5', '4.5.7', '4.6', '4.6.7', '4.6.8', '4.7', '4.7.10', '4.7.10.0', '4.7.11', '4.7.9', '4.8', '4.8.11', '4.9', '5', '5.10', '5.11', '5.6', '5.7', '5.7.10', '5.7.11', '5.7.9', '5.7.9.10.0', '5.8', '5.8.0', '5.8.10', '5.8.11', '5.9',

### Codificar las notas de las melodías

In [12]:
encoded = np.array([note2idx[n] for n in notes])
print(encoded)

[ 69 170 227 ... 178  38 177]


### Crear secuencias temporales

In [13]:
seq_length = 128
char_dataset = Dataset.from_tensor_slices(encoded)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

BATCH_SIZE = 128
BUFFER_SIZE = 1000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Construir la red neuronal con capas GRU

In [18]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return Sequential([
        Input(batch_shape=(batch_size, None)),
        Embedding(vocab_size, embedding_dim),
        GRU(rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'),
        GRU(rnn_units // 2, return_sequences=True, stateful=True,
            recurrent_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])

### Compilar y entrenar el modelo

In [ ]:
def loss = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

def loss(labels, logits):
    return losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch:02d}.weights.h5")

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

SyntaxError: expected '(' (1342273859.py, line 1)

### Generar una nueva melodía

In [16]:
gen_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
checkpoints = glob.glob(os.path.join(checkpoint_dir, '*.weights.h5'))

if checkpoints:
    latest = max(checkpoints, key=os.path.getctime)
    print(f'Último checkpint encontrado: {latest}')
    gen_model.load_weights(latest)

else:
    print('No se encontraron checkpoints. Por favor, entrena el modelo primero.')

NameError: name 'checkpoint_dir' is not defined

In [ ]:
def generate_melody(model, num_generate = seq_length):
    input_eval = [random.choice(range(vocab_size))]
    input_eval = tf.expand_dims(input_eval, 0)
    melody_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions[-1] / temperature
        predicted_id = tf.random.categorical(tf.expand_dims(predictions, 0), num_samples=1)[-1,0].numpy()

        melody_generated.append(idx2note[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)

    return melody_generated

melody = generate_melody(gen_model)
print(melody)

In [ ]:
output_midi = []
midi_file = 'generated_music.mid'

for pattern in melody:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes_ = []
        new_chord = chord.Chord(notes)
        output_midi.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.storedInstrument = instrument.Piano()
        output_midi.append(new_note)